# クラスタリング


[livedoor ニュースコーパス - ITライフハック](https://www.rondhuit.com/download.html#ldcc)

In [17]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [18]:
import os
import re
import sys


pwd = os.getcwd()
path = re.search('.+/自然言語処理', pwd).group(0)

sys.path.append(path)

## テキストの読み込み

In [38]:
import glob
from tqdm import tqdm


document_list = []
for file_name in tqdm(glob.glob('../data/raw/it-life-hack/*.txt')):
    doc = open(file_name, 'r').read()
    document_list.append(doc)

100%|██████████| 410/410 [00:00<00:00, 2460.67it/s]


In [39]:
document_list[0]

'http://news.livedoor.com/article/detail/6793390/\n2012-07-26T13:00:00+0900\nUltrabookをパワーアップ！\u3000mSATA対応の小型SSDがマイクロンより登場\nマイクロンジャパンは、従来の「Crucial m4 SSD」の性能と信頼性をそのまま維持しつつも、1/8のサイズまで超小型化したSSD「Crucial m4 mSATA SSD」を発売する。最近流行のUltrabookや過般性の高いモバイル機器でmSATA対応のガジェットであれば「Crucial m4 mSATA SSD」に乗せ換えることが可能だ。実売想定価格は32GBで5千100円、64GBが7千200円、128GBが1万1千800円、256GBが2万3千800円と、値段もそこそこ手ごろな価格に設定されている。出荷は7月下旬より。\n\n最近では急速に普及しつつあるUltrabookだが搭載するSSDの容量が少ないと嘆いているような人もいるだろう。かといってBTOで購入時に大容量モデルを選択すると予算をオーバーしてしまうようなケースでは、間に合う容量でとりあえず購入しておいて「Crucial m4 mSATA SSD」を後から装着するという方法もイレギュラーながら使える方法だろう。\n\n\n \nmSATA対応のUltrabookなら差し替えるだけでOK \n\n\n\n \nデスクトップ用マザーでもmSATA対応なら利用できる \n\n\nまた、Ultrabookだけでなく、ノートPCでmSATAインターフェイスが余っているというようなケースでは、それを利用してストレージ容量を増やすということもできる。既存のハードディスクの性能を向上させるキャッシュとしても使用可能。Crucial m4 mSATA SSDは、Intel スマート・レスポンス・テクノロジーおよびNVELO製キャッシングソフトウェアDataplexTM に対応しているので既存のHDDと組み合わせることでパフォーマンスをアップさせることも可能だ。\n\n◆製品情報\nメーカー：マイクロンジャパン\n製品名：Crucial m4 mSATA SSD \n型番：CT032M4SSD3、CT064M4SSD3、CT128M4SSD3、CT256M4SSD3\

## 前処理

 1. テキストのクリーニング
     - URLの除去
     - メンションの除去
     - いくつかの記号の除去
 2. 単語の分割(形態素解析)
 3. 単語の正規化
     - 文字種の統一
     - 数字の置き換え
     - 見出し語化
 4. ストップワードの除去
 5. 文書のベクトル表現

### テキストのクリーニング

In [40]:
from src.preprocessings.cleaning import clean_text, clean_html_and_js_tags, clean_url

def clearning(text):
    """テキストのクリーニング."""
    text = clean_text(text)
    text = clean_html_and_js_tags(text)
    text = clean_url(text)
    return text

In [41]:
for i in tqdm(range(len(document_list))):
    document_list[i] = clearning(document_list[i])

100%|██████████| 410/410 [00:00<00:00, 4445.12it/s]


In [42]:
document_list[0]

'ultrabookをパワーアップ！ msata対応の小型ssdがマイクロンより登場マイクロンジャパンは、従来の「crucial m4 ssd」の性能と信頼性をそのまま維持しつつも、1/8のサイズまで超小型化したssd「crucial m4 msata ssd」を発売する。最近流行のultrabookや過般性の高いモバイル機器でmsata対応のガジェットであれば「crucial m4 msata ssd」に乗せ換えることが可能だ。実売想定価格は32gbで5千100円、64gbが7千200円、128gbが1万1千800円、256gbが2万3千800円と、値段もそこそこ手ごろな価格に設定されている。出荷は7月下旬より。最近では急速に普及しつつあるultrabookだが搭載するssdの容量が少ないと嘆いているような人もいるだろう。かといってbtoで購入時に大容量モデルを選択すると予算をオーバーしてしまうようなケースでは、間に合う容量でとりあえず購入しておいて「crucial m4 msata ssd」を後から装着するという方法もイレギュラーながら使える方法だろう。msata対応のultrabookなら差し替えるだけでokデスクトップ用マザーでもmsata対応なら利用できるまた、ultrabookだけでなく、ノートpcでmsataインターフェイスが余っているというようなケースでは、それを利用してストレージ容量を増やすということもできる。既存のハードディスクの性能を向上させるキャッシュとしても使用可能。crucial m4 msata ssdは、intel スマート・レスポンス・テクノロジーおよびnvelo製キャッシングソフトウェアdataplextm に対応しているので既存のhddと組み合わせることでパフォーマンスをアップさせることも可能だ。◆製品情報メーカー：マイクロンジャパン製品名：crucial m4 msata ssd型番：ct032m4ssd3、ct064m4ssd3、ct128m4ssd3、ct256m4ssd3容量： 32gb/64gb/128gb/256gbサイズ：29.83 幅 ×50.80 奥行き ×3.8 高さ mm重さ：10gインターフェイス：sata 6gb/秒 sata 3gb/秒互換 連続読取速度：最大500mb/秒 64gb〜256gbモデル

### 単語の分割(形態素解析)

In [43]:
from src.preprocessings.tokenizer import MeCabTokenizer


tokenizer_neologd = MeCabTokenizer('/usr/local/lib/mecab/dic/mecab-ipadic-neologd')

words_list = []
for doc in tqdm(document_list):
    words = tokenizer_neologd.wakati_baseform(doc)
    words_list.append(words)

100%|██████████| 410/410 [00:06<00:00, 62.51it/s]


In [45]:
words_list[0]

['Ultrabook',
 'を',
 'パワーアップ',
 '！',
 'mSATA',
 '対応',
 'の',
 '小型',
 'SSD',
 'が',
 'マイクロン',
 'より',
 '登場',
 'マイクロンジャパン',
 'は',
 '、',
 '従来',
 'の',
 '「',
 'Crucial',
 'M4',
 'SSD',
 '」',
 'の',
 '性能',
 'と',
 '信頼性',
 'を',
 'そのまま',
 '維持',
 'する',
 'つつ',
 'も',
 '、',
 '1',
 '/',
 '8',
 'の',
 'サイズ',
 'まで',
 '超小型',
 '化',
 'する',
 'た',
 'SSD',
 '「',
 'Crucial',
 'M4',
 'mSATA',
 'SSD',
 '」',
 'を',
 '発売',
 'する',
 '。',
 '最近',
 '流行',
 'の',
 'Ultrabook',
 'や',
 '過般',
 '性',
 'の',
 '高い',
 'モバイル',
 '機器',
 'で',
 'mSATA',
 '対応',
 'の',
 'Gadget',
 'だ',
 'ある',
 'ば',
 '「',
 'Crucial',
 'M4',
 'mSATA',
 'SSD',
 '」',
 'に',
 '乗せる',
 '換える',
 'こと',
 'が',
 '可能',
 'だ',
 '。',
 '実',
 '売',
 '想定',
 '価格',
 'は',
 '32GB',
 'で',
 '5',
 '千',
 '100円',
 '、',
 '64GB',
 'が',
 '7',
 '千',
 '200円',
 '、',
 '128GB',
 'が',
 '1',
 '万',
 '1',
 '千',
 '800円',
 '、',
 '256GB',
 'が',
 '2',
 '万',
 '3',
 '千',
 '800円',
 'と',
 '、',
 '値段',
 'も',
 'そこそこ',
 '手ごろ',
 'だ',
 '価格',
 'に',
 '設定',
 'する',
 'れる',
 'て',
 'いる',
 '。',
 '出荷',
 'は',
 '7月下旬',
 'より',

### 単語の正規化

In [47]:
from src.preprocessings.normalization import normalize, lemmatize_term

normalized_words_list = []
for words in tqdm(words_list):
    words = [normalize(word) for word in words]
#     words = [lemmatize_term(word) for word in words]
    normalized_words_list.append(words)

100%|██████████| 410/410 [00:00<00:00, 602.80it/s]


In [48]:
normalized_words_list[0]

['ultrabook',
 'を',
 'パワーアップ',
 '!',
 'msata',
 '対応',
 'の',
 '小型',
 'ssd',
 'が',
 'マイクロン',
 'より',
 '登場',
 'マイクロンジャパン',
 'は',
 '、',
 '従来',
 'の',
 '「',
 'crucial',
 'm0',
 'ssd',
 '」',
 'の',
 '性能',
 'と',
 '信頼性',
 'を',
 'そのまま',
 '維持',
 'する',
 'つつ',
 'も',
 '、',
 '0',
 '/',
 '0',
 'の',
 'サイズ',
 'まで',
 '超小型',
 '化',
 'する',
 'た',
 'ssd',
 '「',
 'crucial',
 'm0',
 'msata',
 'ssd',
 '」',
 'を',
 '発売',
 'する',
 '。',
 '最近',
 '流行',
 'の',
 'ultrabook',
 'や',
 '過般',
 '性',
 'の',
 '高い',
 'モバイル',
 '機器',
 'で',
 'msata',
 '対応',
 'の',
 'gadget',
 'だ',
 'ある',
 'ば',
 '「',
 'crucial',
 'm0',
 'msata',
 'ssd',
 '」',
 'に',
 '乗せる',
 '換える',
 'こと',
 'が',
 '可能',
 'だ',
 '。',
 '実',
 '売',
 '想定',
 '価格',
 'は',
 '0gb',
 'で',
 '0',
 '千',
 '0円',
 '、',
 '0gb',
 'が',
 '0',
 '千',
 '0円',
 '、',
 '0gb',
 'が',
 '0',
 '万',
 '0',
 '千',
 '0円',
 '、',
 '0gb',
 'が',
 '0',
 '万',
 '0',
 '千',
 '0円',
 'と',
 '、',
 '値段',
 'も',
 'そこそこ',
 '手ごろ',
 'だ',
 '価格',
 'に',
 '設定',
 'する',
 'れる',
 'て',
 'いる',
 '。',
 '出荷',
 'は',
 '0月下旬',
 'より',
 '。',
 '最近',


### ストップワードの除去

In [54]:
from src.preprocessings.stopwords import maybe_download, create_stopwords

maybe_download('../data/processed/stop_word.txt')
stop_words = create_stopwords('../data/processed/stop_word.txt')

File already exists.


In [55]:
stop_words

['あそこ',
 'あたり',
 'あちら',
 'あっち',
 'あと',
 'あな',
 'あなた',
 'あれ',
 'いくつ',
 'いつ',
 'いま',
 'いや',
 'いろいろ',
 'うち',
 'おおまか',
 'おまえ',
 'おれ',
 'がい',
 'かく',
 'かたち',
 'かやの',
 'から',
 'がら',
 'きた',
 'くせ',
 'ここ',
 'こっち',
 'こと',
 'ごと',
 'こちら',
 'ごっちゃ',
 'これ',
 'これら',
 'ごろ',
 'さまざま',
 'さらい',
 'さん',
 'しかた',
 'しよう',
 'すか',
 'ずつ',
 'すね',
 'すべて',
 'ぜんぶ',
 'そう',
 'そこ',
 'そちら',
 'そっち',
 'そで',
 'それ',
 'それぞれ',
 'それなり',
 'たくさん',
 'たち',
 'たび',
 'ため',
 'だめ',
 'ちゃ',
 'ちゃん',
 'てん',
 'とおり',
 'とき',
 'どこ',
 'どこか',
 'ところ',
 'どちら',
 'どっか',
 'どっち',
 'どれ',
 'なか',
 'なかば',
 'なに',
 'など',
 'なん',
 'はじめ',
 'はず',
 'はるか',
 'ひと',
 'ひとつ',
 'ふく',
 'ぶり',
 'べつ',
 'へん',
 'ぺん',
 'ほう',
 'ほか',
 'まさ',
 'まし',
 'まとも',
 'まま',
 'みたい',
 'みつ',
 'みなさん',
 'みんな',
 'もと',
 'もの',
 'もん',
 'やつ',
 'よう',
 'よそ',
 'わけ',
 'わたし',
 'ハイ',
 '上',
 '中',
 '下',
 '字',
 '年',
 '月',
 '日',
 '時',
 '分',
 '秒',
 '週',
 '火',
 '水',
 '木',
 '金',
 '土',
 '国',
 '都',
 '道',
 '府',
 '県',
 '市',
 '区',
 '町',
 '村',
 '各',
 '第',
 '方',
 '何',
 '的',
 '度',
 '文',
 '者',
 '性',
 '体',
 '人',
 '他',
 '今',
 

上記のストップワードを用いてTF-IDFベクトルを作る

### 文書のベクトル化

In [58]:
document_list = [' '.join(words) for words in normalized_words_list]

In [59]:
document_list[0]

'ultrabook を パワーアップ ! msata 対応 の 小型 ssd が マイクロン より 登場 マイクロンジャパン は 、 従来 の 「 crucial m0 ssd 」 の 性能 と 信頼性 を そのまま 維持 する つつ も 、 0 / 0 の サイズ まで 超小型 化 する た ssd 「 crucial m0 msata ssd 」 を 発売 する 。 最近 流行 の ultrabook や 過般 性 の 高い モバイル 機器 で msata 対応 の gadget だ ある ば 「 crucial m0 msata ssd 」 に 乗せる 換える こと が 可能 だ 。 実 売 想定 価格 は 0gb で 0 千 0円 、 0gb が 0 千 0円 、 0gb が 0 万 0 千 0円 、 0gb が 0 万 0 千 0円 と 、 値段 も そこそこ 手ごろ だ 価格 に 設定 する れる て いる 。 出荷 は 0月下旬 より 。 最近 で は 急速 に 普及 する つつ ある ultrabook だ が 搭載 する ssd の 容量 が 少ない と 嘆く て いる よう だ 人 も いる だ う 。 か と いう て bto で 購入 時 に 大 容量 モデル を 選択 する と 予算 を オーバー する て しまう よう だ ケース で は 、 間に合う 容量 で とりあえず 購入 する て おく て 「 crucial m0 msata ssd 」 を 後 から 装着 する という 方法 も イレギュラー ながら 使える 方法 だ う 。 msata 対応 の ultrabook だ 差し替える だけ で ok desktop 用 マザー で も msata 対応 だ 利用 できる また 、 ultrabook だけ だ ない 、 ノート pc で msata interface が 余る て いる という よう だ ケース で は 、 それ を 利用 する て ストレージ 容量 を 増やす という こと も できる 。 既存 の ハードディスク の 性能 を 向上 する せる キャッシュ として も 使用可能 。 crucial m0 msata ssd は 、 intel スマート ・ レスポンス ・ テクノロジー および nvel

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=stop_words)
vecs = vectorizer.fit_transform(document_list)

vecs.toarray()[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [77]:
np.sum(vecs.toarray(), axis=1)

array([ 7.27614521,  5.94135695,  8.18261946,  7.43669201,  9.04471919,
       11.1807058 ,  8.78667383, 12.2926794 ,  6.74021303,  7.88708101,
       11.94422983,  8.05161975,  5.65653367, 12.21587039,  7.5391649 ,
        9.77567545,  9.69974701,  9.54600371, 10.04737874, 10.27525969,
        6.94878741, 12.3302314 ,  7.80086143,  7.5009723 ,  7.41955234,
        6.58215647,  9.63430155,  8.18689572,  7.5165401 ,  8.43942646,
        5.72030365,  8.40699887,  5.96506219,  8.42323579,  6.74734198,
        7.18306668, 12.08090407, 10.63175233,  9.60007134, 10.72592125,
       10.24617732,  9.30400472,  6.62566132, 11.27673285,  7.38903984,
        6.78361155, 11.2002602 ,  8.41976268,  7.49308908, 10.39654507,
        9.14440408,  9.61717768,  8.79790534,  9.10876727, 10.04800542,
        7.34082471,  7.99711512,  5.87622832,  9.49664884,  7.22437242,
        6.41594233, 11.81367161,  9.35216093, 12.70348222,  6.33844014,
       12.80430929,  9.6031568 ,  6.21420642,  7.98500242,  7.89

---
## クラスタリング

In [79]:
documents_df = pd.DataFrame(vecs.toarray())
documents_df.head()

,0,1,2,3,4,5,6,7,8,9,...,12325,12326,12327,12328,12329,12330,12331,12332,12333,12334
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021678,0.357470,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.039713,0.035063,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.124732,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031800,0.037456,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3, random_state=0)
km.fit(documents_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [83]:
clusters = km.predict(documents_df[:10])

In [88]:
for doc, cls in zip(document_list[:10], clusters):
    print('=====' + str(cls) + '=====')
    print(doc)
    print('\n')

=====1=====
ultrabook を パワーアップ ! msata 対応 の 小型 ssd が マイクロン より 登場 マイクロンジャパン は 、 従来 の 「 crucial m0 ssd 」 の 性能 と 信頼性 を そのまま 維持 する つつ も 、 0 / 0 の サイズ まで 超小型 化 する た ssd 「 crucial m0 msata ssd 」 を 発売 する 。 最近 流行 の ultrabook や 過般 性 の 高い モバイル 機器 で msata 対応 の gadget だ ある ば 「 crucial m0 msata ssd 」 に 乗せる 換える こと が 可能 だ 。 実 売 想定 価格 は 0gb で 0 千 0円 、 0gb が 0 千 0円 、 0gb が 0 万 0 千 0円 、 0gb が 0 万 0 千 0円 と 、 値段 も そこそこ 手ごろ だ 価格 に 設定 する れる て いる 。 出荷 は 0月下旬 より 。 最近 で は 急速 に 普及 する つつ ある ultrabook だ が 搭載 する ssd の 容量 が 少ない と 嘆く て いる よう だ 人 も いる だ う 。 か と いう て bto で 購入 時 に 大 容量 モデル を 選択 する と 予算 を オーバー する て しまう よう だ ケース で は 、 間に合う 容量 で とりあえず 購入 する て おく て 「 crucial m0 msata ssd 」 を 後 から 装着 する という 方法 も イレギュラー ながら 使える 方法 だ う 。 msata 対応 の ultrabook だ 差し替える だけ で ok desktop 用 マザー で も msata 対応 だ 利用 できる また 、 ultrabook だけ だ ない 、 ノート pc で msata interface が 余る て いる という よう だ ケース で は 、 それ を 利用 する て ストレージ 容量 を 増やす という こと も できる 。 既存 の ハードディスク の 性能 を 向上 する せる キャッシュ として も 使用可能 。 crucial m0 msata ssd は 、 intel スマート ・ レスポンス ・ テクノロ